# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': ['sum']}).reset_index()
df.head()

,CustomerID,ProductName,Quantity
,,,sum
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot = pd.pivot_table(df, values='Quantity', index=['ProductName'],
                    columns=['CustomerID'], aggfunc=np.sum, fill_value=0)
pivot.head()

C:\Users\ecorti01\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


sum                                      \
CustomerID                        33    200   264   356   412   464   477     
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0     0   
Appetizer - Sausage Rolls             0     0     0     0     0     0     0   
Apricots - Dried                      1     0     0     0     1     0     0   

                                                     ...                    \
CustomerID                        639   649   669    ... 97697 97753 97769   
ProductName                                          ...                     
Anchovy Paste - 56 G Tube             1     0     0  ...     0    25     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0  ...    25    25     0   
Appetizer - Mushroom Tart             1     0     0  ...    25     0     0   
Appetizer - Sausage Rolls             0     0     0  ...     0     0     0   
Apricots - Dried                      0     0     0  ...     0    25     0   

                                                                             
CustomerID                        97793 97900 97928 98069 98159 98185 98200  
ProductName                                                                  
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0  
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0  
Appetizer - Mushroom Tart             0     0     0     0     0    25     0  
Appetizer - Sausage Rolls             0    25    25    25     0    25     0  
Apricots - Dried                      0     0     0     0     0     0     0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist, squareform
customer_similarity = pd.DataFrame(squareform(pdist(pivot['sum'].T, 'euclidean')))
customer_similarity.columns = pivot['sum'].T.index
customer_similarity.index = pivot['sum'].T.index
customer_similarity.head()

# https://stackoverflow.com/questions/13079563/how-does-condensed-distance-matrix-work-pdist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
def get_similar_customers(column_id,distances_matrix):
    closest_5=distances_matrix[column_id].sort_values(ascending=True)[1:6].reset_index()
    return closest_5
customer = 33
closest_5=get_similar_customers(customer,customer_similarity)
closest_5

,CustomerID,33
0,3535,10.488088
1,264,10.488088
2,3317,10.488088
3,2503,10.630146
4,3305,10.677078


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
records = df[df['CustomerID'].isin(closest_5['CustomerID'])]
records.head()

,CustomerID,ProductName,Quantity
,,,sum
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
records_sort = records.groupby(['ProductName']).sum().sort_values(by=('Quantity','sum') ,ascending=False)['Quantity'].reset_index()
records_sort.head()

,ProductName,sum
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
not_purchased_products = records_sort.merge(data[data['CustomerID']==customer][['ProductName','CustomerID']], left_on='ProductName', right_on='ProductName', how='outer')
not_purchased_products[not_purchased_products['CustomerID'].isnull()][:5]

,ProductName,sum,CustomerID
0,Butter - Unsalted,3.0,NaN
1,Wine - Ej Gallo Sierra Valley,3.0,NaN
3,Soup - Campbells Bean Medley,3.0,NaN
4,Wine - Blue Nun Qualitatswein,3.0,NaN
6,Chicken - Soup Base,2.0,NaN


In [11]:
list(not_purchased_products[not_purchased_products['CustomerID'].isnull()][:5]['ProductName'])

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
dict_recomendaciones={}
for customer in data['CustomerID'].unique():
    closest_5=get_similar_customers(customer,customer_similarity)
    records = df[df['CustomerID'].isin(closest_5['CustomerID'])]
    records_sort = records.groupby(['ProductName']).sum().sort_values(by=('Quantity','sum') ,ascending=False)['Quantity'].reset_index()
    not_purchased_products = records_sort.merge(data[data['CustomerID']==customer][['ProductName','CustomerID']], left_on='ProductName', right_on='ProductName', how='outer')
    dict_customer={customer:list(not_purchased_products[not_purchased_products['CustomerID'].isnull()][:5]['ProductName'])}
    dict_recomendaciones.update(dict_customer)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
results = pd.DataFrame(dict_recomendaciones).T
results.head()

,0,1,2,3,4
61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
customer_similarity = pd.DataFrame(squareform(pdist(pivot['sum'].T, 'chebyshev')))
customer_similarity.columns = pivot['sum'].T.index
customer_similarity.index = pivot['sum'].T.index

dict_recomendaciones={}
for customer in data['CustomerID'].unique():
    closest_5=get_similar_customers(customer,customer_similarity)
    records = df[df['CustomerID'].isin(closest_5['CustomerID'])]
    records_sort = records.groupby(['ProductName']).sum().sort_values(by=('Quantity','sum') ,ascending=False)['Quantity'].reset_index()
    not_purchased_products = records_sort.merge(data[data['CustomerID']==customer][['ProductName','CustomerID']], left_on='ProductName', right_on='ProductName', how='outer')
    dict_customer={customer:list(not_purchased_products[not_purchased_products['CustomerID'].isnull()][:5]['ProductName'])}
    dict_recomendaciones.update(dict_customer)
    
results = pd.DataFrame(dict_recomendaciones).T
results.head()

,0,1,2,3,4
61288,Sauce - Demi Glace,"Hickory Smoke, Liquid",Gloves - Goldtouch Disposable,Beets - Mini Golden,Soup V8 Roasted Red Pepper
77352,"Pepper - White, Ground",Island Oasis - Mango Daiquiri,Lettuce - Treviso,"Beef - Chuck, Boneless",Tahini Paste
40094,Halibut - Fletches,Pants Custom Dry Clean,Corn Meal,Tea - Jasmin Green,Beef - Inside Round
23548,Apricots Fresh,Rosemary - Dry,Wine - Valpolicella Masi,Pecan Raisin - Tarts,"Pasta - Penne, Rigate, Dry"
78981,Grapes - Red,Dc - Frozen Momji,Cookie Chocolate Chip With,Lettuce - Frisee,"Tart Shells - Sweet, 4"
